In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt


class ImagePatcher(object):
    def __init__(self, image, dimensions):
        self.image = image
        self.dimensions = dimensions
        self.N, self.M = dimensions  # N = # of rows, M = # of columns
        self.num_rows, self.num_cols = self.get_patch_rows_and_columns()
        self.image_patch_list = []
        self.hist_list = []

    def get_patch_rows_and_columns(self):
        height, width = self.image.shape

        # Calculate the number of rows and columns for a patch
        num_rows = height // self.N
        num_cols = width // self.M

        return (num_rows, num_cols)

    def split_image(self, dir_name):
        # Iterate through the large image and divide it into smaller sub-images
        for row in range(self.N):
            temp = []
            for col in range(self.M):
                # Calculate the coordinates for each sub-image
                x_start = col * self.num_cols
                x_end = (col + 1) * self.num_cols
                y_start = row * self.num_rows
                y_end = (row + 1) * self.num_rows

                sub_image = self.image[y_start:y_end, x_start:x_end]
                temp.append(sub_image)
                cv.imwrite(
                    f"./results/{dir_name}/image-patches/image-patch-{row + 1}-{col + 1}.jpg",
                    sub_image,
                )

            self.image_patch_list.append(temp)

        return self.image_patch_list

    def merge_image_patches(self, canvas_size):
        height, width = canvas_size
        canvas = np.zeros((height, width), dtype=np.uint8)

        for row in range(self.N):
            for col in range(self.M):
                x_start = col * self.num_cols
                x_end = (col + 1) * self.num_cols
                y_start = row * self.num_rows
                y_end = (row + 1) * self.num_rows
                patch = self.image_patch_list[row][col]

                canvas[y_start:y_end, x_start:x_end] = patch

        return canvas

    def get_histograms(self):
        for row in range(self.N):
            temp = []
            for col in range(self.M):
                patch = self.image_patch_list[row][col]
                hist = cv.calcHist([patch], [0], None, [256], [0, 256])
                print(f"hist-{row + 1}-{col + 1}", hist)
                temp.append(hist)

            self.hist_list.append(temp)

        return self.hist_list

In [58]:
# read all images
image_1 = cv.imread("./Dataset/1.jpg", 0)

# resize all images
image_1 = cv.resize(image_1, (1400, 900))

# Create patches of all the images
image_patch_1 = ImagePatcher(image_1, (5, 5))
image_patch_list_1 = image_patch_1.split_image("image-1")

# task # 1: histograms
# image_patch_histograms_1 = image_patch_1.get_histograms()